In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import libraryWAN as lw
import string
import matplotlib.pylab as plt
from numpy import linalg as LA
%matplotlib
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using matplotlib backend: MacOSX
sample_submission.csv
test.csv
train.csv



## ETL

In [2]:
from sklearn.model_selection import train_test_split

def read_data():
    df = pd.read_csv("input/train.csv")
    print ("Shape of base training File = ", df.shape)
    # Remove missing values and duplicates from training data
    df.drop_duplicates(inplace=True)
    df.dropna(inplace=True)
    print("Shape of base training data after cleaning = ", df.shape)
    return df

df = read_data()
df_train, df_test = train_test_split(df, test_size = 0.02)
df_train = df_train.reset_index(drop=True)
df_train = df_train.reset_index()
df_train = df_train.rename(columns = {'index':'mat_idx'})

print (df_train.head(2))
print (df_test.shape)

Shape of base training File =  (404290, 6)
Shape of base training data after cleaning =  (404288, 6)
   mat_idx      id    qid1    qid2  \
0        0  120517  195457  195458   
1        1  315993  440962  424030   

                                           question1  \
0  What are futuristics project ideas for final y...   
1  Is it a compulsion that less than 60% in 12th ...   

                                           question2  is_duplicate  
0  What are some good projects for final year ele...             0  
1  Is MBA a good career choice for those who have...             0  
(8086, 6)


## Load Pre-Acquired Function Words

In [3]:
data = pd.read_csv('function_words.csv',header=None,names=['fwords'])
fwords = data['fwords'].values.tolist()
fwords_dict = {}
for idx, word in enumerate(fwords):
    fwords_dict[word] = idx+1

## Helper Functions

In [4]:
transtable = {ord(c): None for c in string.punctuation}

def processSentence(raw_sentence):
    raw_list = raw_sentence.split(" ")
    clean_list = [str.translate(transtable) for str in raw_list] # remove punctuation
    clean_list = [word.lower().strip() for word in clean_list] # clean the word
    return clean_list

## Similarity Metrics

In [5]:
from sklearn.metrics.pairwise import cosine_similarity as cs
from sklearn.metrics.pairwise import manhattan_distances as md
from sklearn.metrics.pairwise import euclidean_distances as ed
from sklearn.metrics import jaccard_similarity_score as jsc
from sklearn.neighbors import DistanceMetric
from sklearn.preprocessing import MinMaxScaler

minkowski_dis = DistanceMetric.get_metric('minkowski')

mms_scale_man = MinMaxScaler()
mms_scale_euc = MinMaxScaler()
mms_scale_mink = MinMaxScaler()

features = np.zeros((df_train.shape[0], 5))

# Test run for the first 20 lines:
# for ridx, row in df_train.iterrows():
#     if ridx > 20: break
#     else:
        

## Relative Entropy Test

In [9]:
WINDOW_SIZE = 10
ALPHA = 0.8

for idx in range(50):
    s1 = df_train.get_value(idx, 'question1')
    s2 = df_train.get_value(idx, 'question2')
    s1_tok = processSentence(s1)
    s2_tok = processSentence(s2)
    
    wanPair = lw.generateWANPair(s1_tok, s2_tok, WINDOW_SIZE, ALPHA)
    raw1_wan = wanPair['snt1_wan']
    raw2_wan = wanPair['snt2_wan']
        
#     raw1_wan = lw.functionWordWAN(s1_tok, fwords_dict, WINDOW_SIZE, ALPHA)
#     raw2_wan = lw.functionWordWAN(s2_tok, fwords_dict, WINDOW_SIZE, ALPHA)
    
    s1_wan = lw.normalizeWAN(raw1_wan, raw1_wan.shape[0])
    s2_wan = lw.normalizeWAN(raw2_wan, raw1_wan.shape[0])
    
    print(s1)
    print(s2)
    print('Status {}'.format(df_train.get_value(idx, 'is_duplicate')))
    data = lw.relativeEntropy(s1_wan, s2_wan)
    print(data['entropies'])

What are futuristics project ideas for final year electronics engineers?
What are some good projects for final year electronics engineering?
Status 0
[0.0, 0.0]
Is it a compulsion that less than 60% in 12th marks be a factor of failure during your MBA placements?
Is MBA a good career choice for those who have less than 60% aggregate in Engineering? 
Status 0
[4.1553526242505762e-09, 0.0]
What are the top 10 Tamil movies of 2016?
What are your top 10 Tamil songs in 2014?
Status 0
[0.0, 0.0]
What is motivation in human resource management?
What is most frustrating about being a human resource manager?
Status 0
[0.0, 0.0]
Why are porn sites deemed unsafe?
What is an Indian porn site?
Status 0
[0.0, 0.0]
Who is Aung San Suu Kyi?
What can you tell about Aun San Suu Kyi as a person?
Status 0
[0.0, 0.0]
What is it like to be a friend of an introvert?
What does it feel like to be an introvert?
Status 0
[0.0, 0.0]
How do I get my Facebook account?
How do I manage my Facebook account?
Status 0
[